In [1]:
"""
Script for training SAE from a cache of scgpt activations and loading the custom fidelity function.

"""

import warnings
from pathlib import Path
import sys
sys.path.append(str(Path("./").resolve().parent))

import torch
from torch.utils.data import DataLoader

from sae.dictionary import AutoEncoder
from fidelity import get_loss_recovery_fn
from load_sharded_acts import LazyMultiDirectoryTokenDataset
from trainer import StandardTrainer
from training import train_run
from typing import Optional

warnings.filterwarnings("ignore", message="TypedStorage is deprecated")

def get_device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    else:
        return "cpu"


def train_SAE_on_gfm_embeds(
    # Data paths and sources
    gfm_embd_dir: Path,
    eval_seq_path: Optional[Path] = None,
    # Core model architecture
    expansion_factor: int = 8,
    # Training configuration
    batch_size: int = 32,
    steps: int = 2_000,
    seed: int = 0,
    # Optimization parameters
    lr: float = 1e-3,
    warmup_steps: int = 50,
    resample_steps: int = 0,  # 0 to disable
    # Regularization
    l1_penalty: float = 1e-1,
    l1_annealing_pct: float = 0.05,
    # Evaluation settings
    eval_batch_size: int = 128,
    eval_steps: int = 1_000,
    # Logging and checkpointing
    save_dir: str = "models",
    log_steps: int = 100,
    save_steps: int = 50,
    max_ckpts_to_keep: int = 3,
    # Weights & Biases configuration
    use_wandb: bool = True,
    wandb_entity: str = "yunfei-hu-vanderbilt-university",
    wandb_project: str = "test_logging",
    wandb_name: str = "SAE",
):
    """
    Train a Sparse Autoencoder (SAE) using cached activation data from a language model.

    Args:
        # Data paths and sources
        gfm_embd_dir: Directory containing cached model embeddings
        eval_seq_path: Path to sequences for fidelity evaluation, if None, fidelity evaluation is disabled

        # Core model architecture
        expansion_factor: Factor by which to expand the dictionary size relative to input dimension

        # Training configuration
        batch_size: Number of samples per training batch
        steps: Total number of training steps
        seed: Random seed for reproducibility

        # Optimization parameters
        lr: Learning rate for optimizer
        warmup_steps: Number of warmup steps for learning rate scheduler
        resample_steps: Steps between dictionary resampling (0 to disable)

        # Regularization
        l1_penalty: Coefficient for L1 regularization
        l1_annealing_pct: Percentage of training during which to anneal L1 penalty

        # Evaluation settings
        eval_batch_size: Batch size for evaluation
        eval_steps: Frequency of evaluation steps

        # Logging and checkpointing
        save_dir: Directory to save model checkpoints and outputs
        log_steps: Frequency of logging
        save_steps: Frequency of saving checkpoints

        # Weights & Biases configuration
        use_wandb: Whether to use Weights & Biases logging
        wandb_entity: W&B username or team name
        wandb_project: W&B project name
        wandb_name: W&B run name
    """
    device = get_device()

    def collate_fn(batch):
        return torch.stack(batch).to(device)

    # Initialize dataset and dataloader
    acts_dataset = LazyMultiDirectoryTokenDataset(gfm_embd_dir)

    # Determine layer from dataset metadata
    layer = acts_dataset.datasets[0]["layer"]
    plm_name = acts_dataset.datasets[0]["plm_name"]
    print(f"Using activations from layer {layer} of {plm_name}")

    dataloader = DataLoader(
        acts_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
    )
    print(f"Loaded dataset with {len(acts_dataset):,} tokens")

    # Configure resampling
    if resample_steps == 0:
        resample_steps = None

    # Setup trainer configuration
    trainer = StandardTrainer(
        activation_dim=acts_dataset.d_model,
        dict_size=acts_dataset.d_model * expansion_factor,
        warmup_steps=warmup_steps,
        resample_steps=resample_steps,
        lr=lr,
        l1_penalty=l1_penalty,
        l1_annealing_pct=l1_annealing_pct,
        seed=seed,
        wandb_name=wandb_name,
        layer=layer,
        plm_name=plm_name,
        device=device,
        steps=min(steps, len(dataloader)),
    )
    print(f"Training with config: {trainer.config}")

    # Initialize fidelity function if evaluation sequences provided
    if eval_seq_path is not None:
        fidelity_fn = get_loss_recovery_fn(
            esm_model_name=plm_name,
            layer_idx=int(layer),
            eval_seq_path=eval_seq_path,
            device=device,
            batch_size=eval_batch_size,
        )
    else:
        fidelity_fn = None

    # Train the SAE
    train_run(
        # Core training components
        data=dataloader,
        trainer=trainer,
        # Evaluation settings
        fidelity_fn=fidelity_fn,
        eval_steps=eval_steps,
        # Logging and checkpointing
        save_dir=save_dir,
        log_steps=log_steps,
        save_steps=save_steps,
        max_ckpts_to_keep=3,
        # Weights & Biases configuration
        use_wandb=use_wandb,
        wandb_entity=wandb_entity,
        wandb_project=wandb_project,
        additional_wandb_args={
            "eval_seq_path": eval_seq_path,
            "eval_steps": eval_steps,
            "batch_size": batch_size,
            "save_dir": save_dir,
        },
    )

    # print(save_dir)

In [4]:
# Loop through all 12 layers and train SAE on each
for layer in range(12):
    gfm_embd_dir = f"/maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/activations/layer_{layer}"
    save_dir = f"/maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/sae_latents/sae_output_layer{layer+1}"
    
    print(f"=== Training SAE on layer {layer} embeddings ===")
    train_SAE_on_gfm_embeds(
        gfm_embd_dir=gfm_embd_dir,
        save_dir=save_dir,
        wandb_entity="yunfei-hu-vanderbilt-university",
        steps=5000
    )
    print(f"Finished training SAE on layer {layer}, results saved to {save_dir}\n")


=== Training SAE on layer 0 embeddings ===
Loading dataset metadata


100%|██████████| 60/60 [00:00<00:00, 6452.28it/s]

Using activations from layer layer_0 of scgpt
Loaded dataset with 30,588,064 tokens


Training with config: {'dict_class': 'AutoEncoder', 'trainer_class': 'StandardTrainer', 'activation_dim': 512, 'dict_size': 4096, 'lr': 0.001, 'l1_penalty': 0.1, 'l1_annealing_steps': 250, 'steps': 5000, 'warmup_steps': 50, 'resample_steps': None, 'device': 'cuda', 'layer': 'layer_0', 'plm_name': 'scgpt', 'wandb_name': 'SAE', 'submodule_name': None}


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yunfei-hu (yunfei-hu-vanderbilt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Using activations from layer layer_11 of scgpt
Loaded dataset with 30,588,064 tokens
Training with config: {'dict_class': 'AutoEncoder', 'trainer_class': 'StandardTrainer', 'activation_dim': 512, 'dict_size': 4096, 'lr': 0.001, 'l1_penalty': 0.1, 'l1_annealing_steps': 250, 'steps': 5000, 'warmup_steps': 50, 'resample_steps': None, 'device': 'cuda', 'layer': 'layer_11', 'plm_name': 'scgpt', 'wandb_name': 'SAE', 'submodule_name': None}


100%|██████████| 5000/5000 [00:10<00:00, 455.24it/s]

Stopped training because reached max specified steps


act_mean,▆▆▅▆▆██▇█▆▆▇▇▆▆▄█▆▃▆▃▇█▄▆▁▆▅▃▅▅▄▄▆█▅▄▆▅▃
act_std,▅▁▅▂▂▃▄▂▄▂▄▅▆▅▅▃▄▄█▄▂▃▅▃▄▃▄▂▂▆▁▃▂▂▂▃▃▇▂▁
frac_variance_explained,▁▆▇▇▇▇▇▇▇▇▇▇▇▇▇██▇█ ▇▇█ █ █ ▇▇█
l0,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l0_pct_nonzero,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l1_penalty,▁▄██████████████████████████████████████
l2_loss,█▃▃▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▁▂▂▂▁▁▂▂▁▁▁▁▂▂▂▂▂▂▂▂▁▂
loss,█▃▃▃▂▂▃▂▂▃▃▃▂▂▂▂▂▂▂▁▂▂▂▂▁▂▁▁▁▁▂▂▂▂▂▂▂▁▁▂
lr,▁███████████████████████████████████████
mse_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
reconstruction_mean,█▁▇▁▄▄▃▄▅▅▃▃▆▅▅▆▄▄▄▄▄▄▄▅▄▃▅▄▄▅▅▄▄▄▅▄▅▆▅▅


Finished training SAE on layer 11, results saved to /maiziezhou_lab2/yunfei/Projects/interpTFM/activations_cosmx_lung_cancer/sae_latents/sae_output_layer12



In [ ]:
# train_SAE_on_gfm_embeds(gfm_embd_dir='/maiziezhou_lab2/yunfei/Projects/FM_temp/InterPLM/interplm/scgpt/activations/layer_4', 
#                         save_dir='/maiziezhou_lab2/yunfei/Projects/FM_temp/interGFM/sae/sae_output_layer4',
#                         wandb_entity="yunfei-hu-vanderbilt-university",
#                         steps=5000)